#   IAU Zadanie - III. fáza

### Autori: Peter Brandajsky - 50%, Frederik Duvač - 50%

#### Kedze bolo predosle zadanie chaoticky napisane ohladom toho co mame exportnut, tak sme upravili export kde transformujeme aj testovaci dataset pomocou nasej pipeline. Upraveny kod vyzeral takto:


In [ ]:
'''
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import QuantileTransformer, PowerTransformer, StandardScaler, MinMaxScaler, FunctionTransformer
import pandas as pd

# Definícia stĺpcov
normal_columns = [
    'c.android.gm', 'c.android.chrome', 'c.dogalize', 'c.katana',
    'c.android.youtube', 'p.android.documentsui', 'p.system',
    'p.android.chrome', 'p.android.externalstorage', 'p.android.gm',
    'p.android.packageinstaller'
]
skewed_column = ['p.olauncher']

X_train = train_data.drop(columns=['mwra'])
y_train = train_data['mwra']
X_test = test_data.drop(columns=['mwra'])
y_test = test_data['mwra']

# Krok 1: Quantile Transformácia pre šikmý stĺpec
quantile_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (QuantileTransformer(output_distribution="normal", random_state=42), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 2: Power Transformácia pre normálne stĺpce
power_transformer = Pipeline(steps=[
    ('transform', make_column_transformer(
        (PowerTransformer(method='yeo-johnson', standardize=True), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 3: Standard Scaling pre normálne stĺpce
standard_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (StandardScaler(), normal_columns),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Krok 4: Min-Max Scaling pre šikmý stĺpec
minmax_scaler = Pipeline(steps=[
    ('transform', make_column_transformer(
        (MinMaxScaler(), skewed_column),
        remainder='passthrough')),
    ('to_dataframe', FunctionTransformer(lambda x: pd.DataFrame(x, columns=X_train.columns)))
])

# Vytvorenie hlavnej Pipeline s každým krokom zvlášť
pipeline = Pipeline(steps=[
    ('quantile', quantile_transformer),
    ('power', power_transformer),
    ('scaler', standard_scaler),
    ('minmax', minmax_scaler),
], verbose=True)

# Aplikácia pipeline na trénovacie dáta bez 'mwra'
X_train_transformed = pipeline.fit_transform(X_train)
X_test_transformed = pipeline.transform(X_test)

# Spojenie transformovaných dát s cieľovým stĺpcom 'mwra'
train_transformed = pd.concat([y_train.reset_index(drop=True), X_train_transformed], axis=1)
test_transformed = pd.concat([y_test.reset_index(drop=True), pd.DataFrame(X_test_transformed, columns=X_test.columns)], axis=1)

train_transformed
test_transformed

train_transformed.to_csv('dataset/train_transformed.csv', sep='\t', index=False, encoding='utf-8')
test_transformed.to_csv('dataset/test_transformed.csv', sep='\t', index=False, encoding='utf-8')

train_data.to_csv('dataset/train_data.csv', sep='\t', index=False, encoding='utf-8')
test_data.to_csv('dataset/test_data.csv', sep='\t', index=False, encoding='utf-8')
'''

# Načítanie údajov z datasetu


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report

train_data = pd.read_csv('dataset/train_transformed.csv', sep='\t')
test_data = pd.read_csv('dataset/test_transformed.csv', sep='\t')

In [ ]:
print("Train Data:")
print(train_data.head())
print("\nTest Data:")
print(test_data.head())

# Kontrola rozmerov
print(f"Train Data Shape: {train_data.shape}")
print(f"Test Data Shape: {test_data.shape}")

In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:
# Rozdelenie na atributy a cielovu premennu
X_train = train_data.drop(columns=['mwra'])
y_train = train_data['mwra']

X_test = test_data.drop(columns=['mwra'])
y_test = test_data['mwra']

# Vyber features s top korelaciou z fazy 2 - moznosti: 'all', 'top_3', 'top_5', 'top_9'
selected_features = 'all'

top_3_features = ['p.android.chrome', 'c.katana', 'p.system']
top_5_features = ['p.android.chrome', 'c.katana', 'p.system', 'c.android.chrome', 'p.android.gm']
top_9_features = ['p.android.chrome', 'c.katana', 'p.system', 'c.android.chrome', 'p.android.gm', 'c.android.youtube',
                  'c.dogalize', 'p.android.externalstorage', 'c.android.gm']

# Prisposobenie X_train a X_test podla vyberu
if selected_features == 'all':
    pass  # Vsetky stlpce v X_train a X_test
elif selected_features == 'top_3':
    X_train = X_train[top_3_features]
    X_test = X_test[top_3_features]
elif selected_features == 'top_5':
    X_train = X_train[top_5_features]
    X_test = X_test[top_5_features]
elif selected_features == 'top_9':
    X_train = X_train[top_9_features]
    X_test = X_test[top_9_features]
else:
    raise ValueError("Neznáma možnosť pre výber features: vyberte 'all', 'top_3' alebo 'top_5'.")

# Vystup informacii o pouzitych vlastnostiach
print("Použité features:", X_train.columns)

# Kontrola rozmerov
print(f"X_train Shape: {X_train.shape}, y_train Shape: {y_train.shape}")
print(f"X_test Shape: {X_test.shape}, y_test Shape: {y_test.shape}")

- Pri možnosti all (12 atribútov), čiže všetkych ktoré sme exportli z fazy 2 sme dostali najlepši výsledok.
- Tieto modely sú schopné zohľadniť viac vzorcov a získať lepšie výsledky, čo znamená, že niektoré z týchto pridaných atribútov poskytujú dôležité informácie, ktoré pomáhajú pri rozlíšení medzi triedami.

In [ ]:
# Kontrola distribucie tried
print("Distribúcia v trénovacom datasete:")
print(y_train.value_counts(normalize=True))

print("\nDistribúcia v testovacom datasete:")
print(y_test.value_counts(normalize=True))

# 3.1 Jednoduchý klasikátor na základe závislosti v dátach

## 3.1.A Implementácia ID3 klasifikátora

In [ ]:
# Rekurzivne rozdelenie dat do podmnozin na zaklade najlepsieho atributu
class DecisionNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value


# Funkcia na vypocet entropie aktualnej datovej mnoziny
def calculate_entropy(y):
    counts = Counter(y)
    total = len(y)
    entropy = -sum((count / total) * np.log2(count / total) for count in counts.values())
    return entropy


# Funkcia na vypocet informacneho zisku pre numericke atributy
def information_gain(X, y, feature):
    parent_entropy = calculate_entropy(y)

    # Pre numericke atributy potrebujeme vypocitat prahove hodnoty
    thresholds = X[feature].sort_values().unique()
    best_gain = 0
    best_threshold = None

    for threshold in thresholds:
        left_y = y[X[feature] <= threshold]
        right_y = y[X[feature] > threshold]

        # Vypocitame vazenu entropiu pre dany prah
        weighted_entropy = (
                (len(left_y) / len(y)) * calculate_entropy(left_y)
                + (len(right_y) / len(y)) * calculate_entropy(right_y)
        )

        gain = parent_entropy - weighted_entropy
        if gain > best_gain:
            best_gain = gain
            best_threshold = threshold

    return best_gain, best_threshold


# Trieda ID3Classifier s podporou numerickych dat
class ID3Classifier:
    def __init__(self, max_depth=2):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y, depth=0):
        if len(set(y)) == 1:
            return DecisionNode(value=y.iloc[0])

        if depth == self.max_depth:
            return DecisionNode(value=y.mode()[0])

        # Najdeme najlepsi atribut a prah
        best_feature = None
        best_threshold = None
        best_gain = -1

        for feature in X.columns:
            gain, threshold = information_gain(X, y, feature)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

        # Ak neexistuje ziadne delenie, vratime list
        if best_feature is None:
            return DecisionNode(value=y.mode()[0])

        node = DecisionNode(feature=best_feature, threshold=best_threshold)
        left_mask = X[best_feature] <= best_threshold
        right_mask = X[best_feature] > best_threshold

        node.left = self.fit(X[left_mask], y[left_mask], depth + 1)
        node.right = self.fit(X[right_mask], y[right_mask], depth + 1)

        return node

    def predict_one(self, x, node):
        if node.value is not None:
            return node.value

        if x[node.feature] <= node.threshold:
            return self.predict_one(x, node.left)
        else:
            return self.predict_one(x, node.right)

    def predict(self, X):
        return X.apply(lambda x: self.predict_one(x, self.tree), axis=1)


# Trenovanie ID3 modelu
id3 = ID3Classifier(max_depth=3)
id3.tree = id3.fit(X_train, y_train)

# Predikcia na trenovacej a testovacej mnozine
y_train_pred_id3 = id3.predict(X_train)
y_test_pred_id3 = id3.predict(X_test)

print("Train predikcie:\n", y_train_pred_id3)
print("Test predikcie:\n", y_test_pred_id3)

## 3.1.B Vyhodnotenie modelu

In [ ]:
print("Výsledky pre trénovaciu množinu:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_id3):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_id3, average='binary'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_id3, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_train, y_train_pred_id3))

print("\nVýsledky pre testovaciu množinu:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_id3):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_id3, average='binary'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred_id3, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_test, y_test_pred_id3))

### Interpretácia výsledkov pre max depth 3 a 12 atributov:
1. Presnosť (Accuracy):
    - Trénovacia presnosť: 86.53 %
    - Testovacia presnosť: 87.07 %
    - Obidve sú veľmi dobré a pomerne stabilné medzi trénovacím a testovacím datasetom, čo naznačuje, že model dobre generalizuje.

2. Presnosť (Precision):
    - Trénovacia presnosť: 87.59 %
    - Testovacia presnosť: 88.26 %
    - Vysoká presnosť na oboch množinách znamená, že model minimalizuje počet falošných pozitív (t.j. nesprávne klasifikované pozitívne prípady).

3. Recall (Revokovanie):
    - Trénovacia revokácia: 91.46 %
    - Testovacia revokácia: 91.53 %
    - Model má veľmi vysokú recall hodnotu, čo znamená, že zachytáva väčšinu skutočných pozitívnych prípadov, čo je dôležité pre odhaľovanie malvéru.

4. F1-Score:
    - F1-Score (pre triedu 1.0): 0.89 (trénovacia) a 0.90 (testovacia)
    - F1-Score je veľmi dobrá metrika pre nevyvážené dáta, ako je náš prípad, kde jedna trieda (1.0) je viac zastúpená. Tento výsledok ukazuje, že model má veľmi dobrý kompromis medzi presnosťou a recallom.

5. Macro a weighted average:
    - Macro avg (priemer medzi triedami):
        - Trénovanie: Precision 0.86, Recall 0.85
        - Testovanie: Precision 0.87, Recall 0.86
    - Weighted avg (vážený priemer podľa počtu príkladov v každej triede):
        - Trénovanie: Precision 0.86, Recall 0.87
        - Testovanie: Precision 0.87, Recall 0.87
    - Tieto hodnoty tiež naznačujú, že model je vyvážený medzi oboma triedami, bez výrazných sklonov.

## 3.1.C Overenie overfittingu

In [ ]:
# Metriky pre trenovaciu a testovaciu mnozinu
train_metrics = [accuracy_score(y_train, y_train_pred_id3), precision_score(y_train, y_train_pred_id3),
                 recall_score(y_train, y_train_pred_id3)]
test_metrics = [accuracy_score(y_test, y_test_pred_id3), precision_score(y_test, y_test_pred_id3),
                recall_score(y_test, y_test_pred_id3)]

metrics = ['Accuracy', 'Precision', 'Recall']

x = np.arange(len(metrics))
width = 0.35

ficomp_g, ax = plt.subplots(figsize=(8, 6))

# Stlpce pre trenovaciu a testovaciu mnozinu
rects1 = ax.bar(x - width / 2, train_metrics, width, label='Train', color='b')
rects2 = ax.bar(x + width / 2, test_metrics, width, label='Test', color='g')

# Pridanie popisov, titulkov a zobrazenie
ax.set_ylabel('Scores')
ax.set_title('Comparison of metrics between Train and Test data')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

plt.tight_layout()
plt.show()

#### Naše výsledky sú veľmi dobré! Máme vysokú presnosť, recall aj F1-skóre, ktoré naznačujú, že model funguje veľmi dobre na detekciu mwra aktivity. Z interpretácie výsledkov a z grafu vidíme, že metriky sú vyvážené a pre testovaciu množinu sú o niečo lepšie, preto náš model nie je preučený.

# 3.2 Trénovanie a vyhodnotenie klasikátorov strojového učenia

## 3.2.A Trénovanie stromového algoritmu - Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(max_depth=3, random_state=42, criterion='entropy')

# model training
dt_clf.fit(X_train, y_train)

# prediction on test and train set
y_train_pred_dt = dt_clf.predict(X_train)
y_test_pred_dt = dt_clf.predict(X_test)

# performance evaluation
print("Decision Tree Classifier - Trénovacia množina:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_dt):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_dt, average='binary'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_dt, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_train, y_train_pred_dt))

print("Decision Tree Classifier - Testovacia množina:")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_dt):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_dt, average='binary'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred_dt, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_test, y_test_pred_dt))

## 3.2.B Trénovanie nestromového algoritmu - Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(random_state=42, max_iter=1000)

# model training
lr_clf.fit(X_train, y_train)

# prediction on test and train set
y_train_pred_lr = lr_clf.predict(X_train)
y_test_pred_lr = lr_clf.predict(X_test)

# performance evaluation
print("Logistic Regression - Trénovacia množina")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred_lr):.4f}")
print(f"Precision: {precision_score(y_train, y_train_pred_lr, average='binary'):.4f}")
print(f"Recall: {recall_score(y_train, y_train_pred_lr, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_train, y_train_pred_lr))

print("Logistic Regression - Testovacia množina")
print(f"Accuracy: {accuracy_score(y_test, y_test_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_test_pred_lr, average='binary'):.4f}")
print(f"Recall: {recall_score(y_test, y_test_pred_lr, average='binary'):.4f}")
print("\nPodrobný report:")
print(classification_report(y_test, y_test_pred_lr))

## 3.2.C Porovnanie s ID3 klasifikátorom na základe Classification Report pre jednotlive triedy 0 a 1

- Pre triedu 1 (pozitívnu) všetky modely dosahujú veľmi podobné výsledky, Logistic Regression disponuje mierne lepším výkonom.
- Pre triedu 0 (negatívnu) Logistic Regression opäť vykazuje lepšie výsledky, najmä čo sa týka precision.

- Toto porovnanie ukazuje, že Logistic Regression má lepší výkon na oboch triedach v porovnaní s ID3 a Decision Tree. ID3 a Decision Tree majú podobné výsledky, ale Logistic Regression je silnejší vo väčšine metrikách.

#### Podrobnejšie metriky a vizualizácia je v bode 3.2.5

## 3.2.D Vizualizácia pravidiel stromového modelu a Logistic Regression

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
tree.plot_tree(dt_clf, filled=True, feature_names=X_train.columns, class_names=['0', '1'], fontsize=12)
plt.title("Decision Tree Visualization")
plt.show()

#### Niektoré vlastnosti, ako documentsui, externalstorage, alebo gm, majú veľký vplyv na rozhodovanie modelu. Sú na vyšších úrovniach stromu, pretože najlepšie rozlišujú medzi triedami.

In [ ]:
# coefficient extraction a sort by influence
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': lr_clf.coef_[0]})
coefficients = coefficients.sort_values(by='Coefficient', ascending=False)

# coefficient visualisation
plt.figure(figsize=(10, 6))
coefficients.plot(kind='bar', x='Feature', y='Coefficient', legend=False, color='skyblue')
plt.title('Logistic Regression Coefficients')
plt.ylabel('Coefficient Value')
plt.tight_layout()
plt.show()

## 3.2.E Vyhodnotenie pomocou Accuracy, Precision, Recall

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Metriky pre každý model
metrics_train = {
    'Model': ['ID3', 'Decision Tree', 'Logistic Regression'],
    'Accuracy': [
        accuracy_score(y_train, y_train_pred_id3),
        accuracy_score(y_train, y_train_pred_dt),
        accuracy_score(y_train, y_train_pred_lr)
    ],
    'Precision': [
        precision_score(y_train, y_train_pred_id3),
        precision_score(y_train, y_train_pred_dt),
        precision_score(y_train, y_train_pred_lr)
    ],
    'Recall': [
        recall_score(y_train, y_train_pred_id3),
        recall_score(y_train, y_train_pred_dt),
        recall_score(y_train, y_train_pred_lr)
    ]
}

metrics_test = {
    'Accuracy': [
        accuracy_score(y_test, y_test_pred_id3),
        accuracy_score(y_test, y_test_pred_dt),
        accuracy_score(y_test, y_test_pred_lr)
    ],
    'Precision': [
        precision_score(y_test, y_test_pred_id3),
        precision_score(y_test, y_test_pred_dt),
        precision_score(y_test, y_test_pred_lr)
    ],
    'Recall': [
        recall_score(y_test, y_test_pred_id3),
        recall_score(y_test, y_test_pred_dt),
        recall_score(y_test, y_test_pred_lr)
    ]
}

# Pretransformovanie do DataFrame
metrics_train_df = pd.DataFrame(metrics_train)
metrics_test_df = pd.DataFrame(metrics_test)

# Vizualizácia všetkých metrík pre trénovacie a testovacie dáta
models = ['ID3', 'Decision Tree', 'Logistic Regression']
metrics = ['Accuracy', 'Precision', 'Recall']

# Nastavenie pre graf
x = np.arange(len(models))
width = 0.15

# Vytvorenie grafu
comp_fig, ax = plt.subplots(figsize=(12, 8))

# Stĺpce
comp_rects1 = ax.bar(x - 2 * width, metrics_train_df['Accuracy'], width, label='Accuracy (Train)', color='green')
comp_rects2 = ax.bar(x - width, metrics_test_df['Accuracy'], width, label='Accuracy (Test)', color='lightgreen')
comp_rects3 = ax.bar(x, metrics_train_df['Precision'], width, label='Precision (Train)', color='skyblue')
comp_rects4 = ax.bar(x + width, metrics_test_df['Precision'], width, label='Precision (Test)', color='lightblue')
comp_rects5 = ax.bar(x + 2 * width, metrics_train_df['Recall'], width, label='Recall (Train)', color='darkred')
comp_rects6 = ax.bar(x + 3 * width, metrics_test_df['Recall'], width, label='Recall (Test)', color='red')

# Pridanie názvov a popisov
ax.set_ylabel('Skóre')
ax.set_title('Porovnanie metrík Accuracy, Precision a Recall medzi modelmi (Trénovacia a Testovacia množina)')
ax.set_xticks(x + width * 0.5)
ax.set_xticklabels(models)
ax.legend(loc='lower right')


# Funkcia pre pridanie hodnôt na stĺpce
def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('%.4f' % height,
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')


# Pridanie hodnôt na stĺpce
add_labels(comp_rects1)
add_labels(comp_rects2)
add_labels(comp_rects3)
add_labels(comp_rects4)
add_labels(comp_rects5)
add_labels(comp_rects6)

# Zobrazenie grafu
plt.tight_layout()
plt.show()

### Výstup:

Z výstupov môžeme vidieť, že Logistic Regression dosiahol najvyššiu presnosť a recall na trénovacej aj testovacej množine. Tento model je najpresnejší a má lepšiu schopnosť zachytiť pozitívne vzory (vysoký recall).

ID3 a Decision Tree majú veľmi podobné výsledky, čo naznačuje, že Decision Tree model (ktorý je optimalizovaný v rámci scikit-learn) poskytuje porovnateľné alebo lepšie výsledky než ručne implementovaný ID3 model. Avšak, medzi ID3 a Decision Tree nie sú výrazné rozdiely, keďže oba modely sú v podstate rozhodovacími stromami.

#### 1. ID3:

- Funguje ako základný model, ktorý má stabilné výsledky, ale mierne zaostáva oproti ostatným modelom.
- Precision, recall a F1-score sú rovnaké pre Decision Tree, čo indikuje podobnú štruktúru.

#### 2. Decision Tree:

- Zlepšenie oproti ID3 v testovacej množine, ale rozdiel je minimálny.
- Lepšia flexibilita algoritmu nevedie k dramatickému zlepšeniu presnosti.

#### 3. Logistic Regression:

- Najvyššie výsledky pre všetky metriky (precision, recall, F1-score) v testovacej množine.
- Model má výhodu v tom, že je vhodný pre tento typ úlohy.

# 3.3 Optimalizácia alias hyperparameter tuning

## 3.3.A

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import matplotlib.pyplot as plt

# 1. Nastavenie parametrov pre grid search
param_grid = {
    'criterion': ['gini', 'entropy'],  
    'max_depth': [3, 5, 7, 10],        
    'min_samples_split': [2, 5, 10],   
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None] 
}

# 2. Inicializácia modelu a GridSearchCV
dt = DecisionTreeClassifier(random_state=42)
grid_search = GridSearchCV(
    estimator=dt,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 3. Trénovanie modelu
print("Trénovanie Grid Search modelu...")
grid_search.fit(X_train, y_train)

# 4. Výsledky a analýza
print("\nNajlepšie parametre:", grid_search.best_params_)
print("Najlepšie skóre:", grid_search.best_score_)

# 5. Predikcie najlepšieho modelu
best_model = grid_search.best_estimator_
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# 6. Vyhodnotenie výkonu
print("\nVýsledky pre trénovaciu množinu:")
print(classification_report(y_train, y_train_pred))

print("\nVýsledky pre testovaciu množinu:")
print(classification_report(y_test, y_test_pred))

# 7. Vizualizácia výsledkov pre rôzne kombinácie parametrov
results = pd.DataFrame(grid_search.cv_results_)

# Graf pre max_depth
plt.figure(figsize=(15, 5))
plt.subplot(1, 2, 1)
depth_scores = {}

for depth in param_grid['max_depth']:
    mask = results['param_max_depth'] == depth
    scores = results[mask]['mean_test_score']
    depth_scores[depth] = {
        'mean': scores.mean(),
        'std': scores.std()
    }

depths = list(depth_scores.keys())
means = [depth_scores[d]['mean'] for d in depths]
stds = [depth_scores[d]['std'] for d in depths]

plt.errorbar(depths, means, yerr=stds, marker='o')
plt.title('Závislosť presnosti od hĺbky stromu')
plt.xlabel('Maximálna hĺbka stromu')
plt.ylabel('Priemerná presnosť')
plt.grid(True)

# Graf pre max_features
plt.subplot(1, 2, 2)
feature_scores = {}

for feature in param_grid['max_features']:
    mask = results['param_max_features'] == feature
    scores = results[mask]['mean_test_score']
    feature_scores[str(feature)] = {
        'mean': scores.mean(),
        'std': scores.std()
    }

features = list(feature_scores.keys())
means = [feature_scores[f]['mean'] for f in features]
stds = [feature_scores[f]['std'] for f in features]

plt.bar(features, means, yerr=stds)
plt.title('Závislosť presnosti od max_features')
plt.xlabel('max_features')
plt.ylabel('Priemerná presnosť')
plt.grid(True)

plt.tight_layout()
plt.show()

# 8. Analýza výsledkov
print("\nAnalýza výkonu pre rôzne nastavenia:")
print("\nPre rôzne hĺbky stromu:")
for depth in depths:
    print(f"Hĺbka {depth}:")
    print(f"Priemerná presnosť: {depth_scores[depth]['mean']:.4f}")
    print(f"Štandardná odchýlka: {depth_scores[depth]['std']:.4f}")

print("\nPre rôzne max_features:")
for feature in features:
    print(f"max_features={feature}:")
    print(f"Priemerná presnosť: {feature_scores[feature]['mean']:.4f}")
    print(f"Štandardná odchýlka: {feature_scores[feature]['std']:.4f}")

# 9. Kontrola underfittingu
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("\nKontrola kvality modelu:")
print(f"Presnosť na trénovacej množine: {train_acc:.4f}")
print(f"Presnosť na testovacej množine: {test_acc:.4f}")
print(f"Rozdiel: {abs(train_acc - test_acc):.4f}")

if train_acc < 0.85:
    print("VAROVANIE: Možný underfitting - model má nízku presnosť na trénovacích dátach")
elif abs(train_acc - test_acc) > 0.05:
    print("VAROVANIE: Možný overfitting - veľký rozdiel medzi trénovacou a testovacou presnosťou")
else:
    print("Model je dobre vyvážený - bez zjavného underfittingu alebo overfittingu")

## 3.3.B

## 3.3.C

## 3.3.D

# 3.4 Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasikáciu

## 3.4.A

## 3.4.B

## 3.4.C

## 3.4.D

## 3.4.E

# Načítanie údajov z datasetu


# 3.1 Jednoduchý klasikátor na základe závislosti v dátach

## 3.1.A

## 3.1.B

## 3.1.C

# 3.2 Trénovanie a vyhodnotenie klasikátorov strojového učenia

## 3.2.A

## 3.2.B

## 3.2.C

## 3.2.D

## 3.2.E

# 3.3 Optimalizácia alias hyperparameter tuning

## 3.3.A

## 3.3.B

## 3.3.C

## 3.3.D

# 3.4 Vyhodnotenie vplyvu zvolenej stratégie riešenia na klasikáciu

## 3.4.A

## 3.4.B

## 3.4.C

## 3.4.D

## 3.4.E